prompt design


In [6]:
from transformers import pipeline
from datetime import datetime
import json


In [8]:
# Load model
llm = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_length=256,
    device=-1  # Set to -1 for CPU, or 0 for GPU if available
)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cpu


In [33]:
def parse_time_response(response_text: str):
    import re

    result = {}

    # Match common date expressions and short/full month names
    date_pattern = r"\b(?:\d{1,2}(?:st|nd|rd|th)?\s+(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|" \
                   r"May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:t)?(?:ember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)|" \
                   r"(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|" \
                   r"Sep(?:t)?(?:ember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\s+\d{1,2}(?:st|nd|rd|th)?|" \
                   r"yesterday|today|tomorrow|last\s+\w+|this\s+\w+)\b"

    # Match time-of-day expressions
    time_of_day_pattern = r"\b(morning|afternoon|evening|night|noon|midnight)\b"

    # Match time ranges (e.g., 3-4 pm)
    time_range_pattern = r"\b\d{1,2}(:\d{2})?\s?(am|pm)?\s?[-–]\s?\d{1,2}(:\d{2})?\s?(am|pm)?\b"

    # Match single time (e.g., 9:30 pm or 8)
    time_pattern = r"\b\d{1,2}(:\d{2})?\s?(am|pm)?\b"

    # Find matches
    date_match = re.search(date_pattern, response_text, re.IGNORECASE)
    time_of_day_match = re.search(time_of_day_pattern, response_text, re.IGNORECASE)
    time_range_match = re.search(time_range_pattern, response_text, re.IGNORECASE)
    time_match = re.search(time_pattern, response_text, re.IGNORECASE)

    # Assign date if found
    if date_match:
        result["date"] = date_match.group().strip()

    # Handle time range if found
    if time_range_match:
        result["time_range"] = time_range_match.group().strip()
    elif time_match:
        time_str = time_match.group().strip()

        # If am/pm not in time, try to infer from time-of-day
        if not re.search(r"\b(am|pm)\b", time_str, re.IGNORECASE) and time_of_day_match:
            tod = time_of_day_match.group().lower()
            if tod in ["evening", "night"]:
                time_str += " pm"
            elif tod in ["morning"]:
                time_str += " am"
            elif tod == "noon":
                time_str = "12:00 pm"
            elif tod == "midnight":
                time_str = "12:00 am"

        result["time"] = time_str

    if not result:
        return {"error": "Could not extract date-time or time"}

    return result


In [34]:
prompt = "Extract the exact datetime or time range from this: Aug 10 evening around 9:30"
response = llm(prompt)[0]['generated_text']
print("📤 Raw response:", response)

parsed_output = parse_time_response(response)
print("🗓️ Parsed JSON:", json.dumps(parsed_output, indent=2))


📤 Raw response: Aug 10 evening around 9:30
🗓️ Parsed JSON: {
  "date": "Aug 10",
  "time": "10 pm"
}
